# MUTRAFF - Congestion Map Drawing
Congestion tracing based on Mutraff Experiments over Google Maps
References:
* Mutraff
* Jupyter gmaps: https://jupyter-gmaps.readthedocs.io/en/v0.3.3/gmaps.html
* Blog examples: https://people.revoledu.com/kardi/tutorial/Python/Displaying+Locations+using+Heatmap.html

## Imports section

In [2]:
import pandas as pd
import numpy as np
import googlemaps
import gmaps
import yaml

# IMPORT CONFIG YAML
gm = googlemaps.Client(key=API_KEY)
gmaps.configure(api_key=API_KEY) # Your Google API key

## Definitions

In [3]:
MUTRAFF_HOME="/Users/alvaro/Desktop/workspace/mutraff/uah-gist-mutraff-bastra"
MUTRAFF_EXP_PATH=MUTRAFF_HOME + "/experiments/tmp"
SIMUL_TIME_STATS=None
COLUMNS=[]
exp={}
label={}

SMALL_FONT_SIZE=10
MID_FONT_SIZE=12
BIG_FONT_SIZE=14

## Functions Library

### Geocode and GeocodeStreetLocationCity functions
In case of receiving just edge names or address, the following functions will do the geocoding of street, location and the city.
The algorithm goes as follow:
* the code will try to search for street, location and city.
* If it is not succesful, it will try to search for street and the city.
* If it is still unsuccessful, it will search for location and city.
* If all of these attempts are not successful, it will produce fail in geocoding. 
* If any of the attempt is successful, it will append the result into the list and return this list of latitude and longitude.

In [4]:
def Geocode(query):
    # do geocoding
    try:
        geocode_result = gm.geocode(query)[0]       
        latitude = geocode_result['geometry']['location']['lat']
        longitude = geocode_result['geometry']['location']['lng']
        return latitude,longitude
    except IndexError:
        return 0
        
def GeocodeStreetLocationCity(data):
    lat=[]                            # initialize latitude list
    lng=[]                            # initialize longitude list
    start = data.index[0]             # start from the first data
    end = data.index[maxRow-1]        # end at maximum number of row
    #end = 100
    for i in range(start,end+1,1):    # iterate all rows in the data
        isSuccess=True                # initial Boolean flag
        
        #query = data.Street[i] + ',' + data.Location[i]
        query = data.Street[i] + ',' + 'Manila' # data.Location[i]
        # query = data.Location[i]
        # result=Geocode(query)
        result = 0
        if result==0:         # if not successful,
            print(i, 'is failed')
            isSuccess = False
        else:
            print(i, result)
        if isSuccess==True:           # if geocoding is successful,
            # store the results
            lat.append(result[0])     # latitude
            lng.append(result[1])     # longitude
    return lat,lng

### DrawHeatMap function
Creates a heat map to represent congestion data

In [5]:
def drawHeatMap( center_address, v_lat_long, v_weigths, zoom, intensity, radius):
    # do geocode for the whole mega city
    geocode_result = gm.geocode(center_address)[0]  # change the name into your city of interest

    # get the center of the city
    center_lat=geocode_result['geometry']['location']['lat']
    center_lng=geocode_result['geometry']['location']['lng']
    # print('center=',center_lat,center_lng)

    # setting the data and parameters
    # heatmap_layer = gmaps.heatmap_layer(locations, val, dissipating = True)
    heatmap_layer = gmaps.heatmap_layer(v_lat_long, v_weigths, dissipating = True)
    # heatmap_layer = gmaps.WeightedHeatmap(v_lat_long)
    heatmap_layer.disspating = True
    heatmap_layer.max_intensity = intensity
    heatmap_layer.point_radius = radius
    # draw the heatmap into a figure
    fig = gmaps.figure()
    fig = gmaps.figure(center = [center_lat,center_lng], zoom_level=zoom)
    fig.add_layer(heatmap_layer)
    return fig

### loadExperiment function
Import csv data from experiments into valuable data

In [6]:
def load_data(experiment):
    global MUTRAFF_EXP_PATH
    
    theFile="edges.csv"
    filename = "{}/{}/{}".format(MUTRAFF_EXP_PATH,experiment,theFile)
    filename="alcalahenares.edges.csv"
    print("Parsing edges catalog "+theFile)
    df1 = pd.read_csv(filename)
    
    theFile="edge_stats.csv"
    filename = "{}/{}/{}".format(MUTRAFF_EXP_PATH,experiment,theFile)
    print("Parsing data traffic file "+theFile)
    df2 = pd.read_csv(filename)
    
    return df1, df2

## Experiments

In [7]:
exp[0]="alcalahenares_M_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit20_11_190508_003728"

exp[2200]="alcalahenares_M3h_nomaps_tele60_timeALL_fulltraffic_190510_235642"
exp[2201]="alcalahenares_M3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit50_01_190511_163910"

exp[2300]="alcalahenares_L3h_nomaps_tele60_timeALL_fulltraffic_190515_130939"
exp[2301]="alcalahenares_L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit50_01_190515_133434"

exp[2400]="alcalahenares_2L3h_nomaps_tele60_timeALL_fulltraffic_190515_150939"
exp[2401]="alcalahenares_2L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit50_01_190515_161514"

exp[2410]="alcalahenares_2L3h_nomaps_tele60_timeALL_fulltraffic_190515_182031"
exp[2411]="alcalahenares_2L3h_mutraff_tele60_uni5x8_timeALL_fulltraffic_logit50_01_190515_191357"
exp[2412]="alcalahenares_2L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit50_01_190515_200544"

exp[2420]="alcalahenares_2L3h_nomaps_tele60_timeALL_fulltraffic_190515_202233"
exp[2421]="alcalahenares_2L3h_mutraff_tele60_uni5x8_timeALL_fulltraffic_logit50_01_190515_205714"
exp[2422]="alcalahenares_2L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit50_01_190515_212938"

exp[2430]="alcalahenares_3L3h_nomaps_tele60_timeALL_fulltraffic_190516_084853"
exp[2431]="alcalahenares_3L3h_mutraff_tele60_uni5x8_timeALL_fulltraffic_logit10_01_190521_091537"
exp[2432]="alcalahenares_3L3h_mutraff_tele60_uni5x8_timeALL_fulltraffic_logit20_01_190521_143019"
exp[2433]="alcalahenares_3L3h_mutraff_tele60_uni5x8_timeALL_fulltraffic_logit50_01_190516_091540"
exp[2434]="alcalahenares_3L3h_mutraff_tele60_uni5x8_timeALL_fulltraffic_logit100_01_190521_210550"

exp[2441]="alcalahenares_3L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit10_01_190521_213112"
exp[2442]="alcalahenares_3L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit20_01_190521_220448"
exp[2443]="alcalahenares_3L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit50_01_190516_114311"
exp[2444]="alcalahenares_3L3h_mutraff_tele60_uni5x16_timeALL_fulltraffic_logit100_01_190521_223020"

In [8]:
label[2430]="Alcala 3L - No MuTraff maps"
label[2431]="Alcala 3L - 8  MuTraff maps - uniform(5) - 10% usage"
label[2432]="Alcala 3L - 8  MuTraff maps - uniform(5) - 20% usage"
label[2433]="Alcala 3L - 8  MuTraff maps - uniform(5) - 50% usage"
label[2434]="Alcala 3L - 8  MuTraff maps - uniform(5) - 100% usage"
label[2441]="Alcala 3L - 16 MuTraff maps - uniform(5) - 10% usage"
label[2442]="Alcala 3L - 16 MuTraff maps - uniform(5) - 20% usage"
label[2443]="Alcala 3L - 16 MuTraff maps - uniform(5) - 50% usage"
label[2444]="Alcala 3L - 16 MuTraff maps - uniform(5) - 100% usage"

### Incident management

In [28]:
exp[2500]="alcalahenares_3L3h_nomaps_tele60_timeALL_fulltraffic_incident_190603_000953"
label[2500]="Alcala 3L - No MuTraff maps - Incident 30' main roundabout"

In [30]:
experiment=exp[2430]
experiment=exp[2500]

## Part 1 : Load traffic data from measures or experiments

In [31]:
edges, stats = load_data( experiment )

Parsing edges catalog edges.csv
Parsing data traffic file edge_stats.csv


In [32]:
edges.head()

,edge_id,odmat_id,from_node,to_node,from_lat,from_long,to_lat,to_long,center_lat,center_long,name
0,1,-100851682,1165482312,1165482328,40.476015,-3.394803,40.476372,-3.393535,40.476194,-3.394169,NaN
1,2,-100851688,1165482328,1165482290,40.476372,-3.393535,40.476832,-3.391995,40.476602,-3.392765,NaN
2,3,-100851690,1165482325,1165482312,40.475548,-3.396469,40.476015,-3.394803,40.475782,-3.395636,NaN
3,4,-100851695,1165482322,1165482272,40.468375,-3.405725,40.469638,-3.401857,40.469007,-3.403791,NaN
4,5,-103584157,311472409,1195985289,40.484626,-3.396309,40.485168,-3.396721,40.484897,-3.396515,NaN


* Latitud: 40.4994044

* Longitud: -3.3401732

In [33]:
stats.head()


,time,edge_id,traf_travel_time,traf_waiting_time,traf_total_veh_num,traf_halted_veh_num,traf_av_occupancy,traf_av_speed,emission_co2,emission_co,emission_hc,emission_noise,emission_nox,emission_PMx,consum_epower,consum_fuel
0,0,0,4.122030,0.0,0.0,0.0,0.0,27.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,5.040677,0.0,0.0,0.0,0.0,27.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2,5.411807,0.0,0.0,0.0,0.0,27.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,3,25.655868,0.0,0.0,0.0,0.0,13.89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,4,4.825054,0.0,0.0,0.0,0.0,13.89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### drawTrafficInstantShot
Prints a heatmap with an spatial and time exposure

In [34]:
def drawInstantShot( city, dataset, timestamp, variable_name ):
    time_shot = dataset[dataset['time']==timestamp].groupby(['time','edge_id']).sum() # .unstack()['traf_halted_veh_num']
    v_lat_long = time_shot[['center_lat','center_long']]
    v_weight = time_shot[variable_name]
    
    print( "Lat/Long size={}".format( len(v_lat_long) ))
    print( "Weight size={}".format( len(v_weight) ))
    # set up parameters
    zoom=13
    max_intensity=float(np.max(v_weight))
    radius=10

    # call the function to draw the heatmap
    drawHeatMap(city, v_lat_long, v_weight, zoom, max_intensity, radius)

merge informacion together

In [35]:
dataset = pd.merge(edges, stats, left_on='edge_id', right_on='edge_id')

In [36]:
time=2520
drawInstantShot( 'Alcala de Henares', dataset, time, 'traf_halted_veh_num' )

Lat/Long size=7828
Weight size=7828


### Join data and extract a time snapshot
Summarize all the data corresponding to the *timestamp* time and combines this data with edge values.

In [46]:
timestamp=3000
timestamp=2520
time_stats_shot  = stats[stats['time']==timestamp].groupby(['time','edge_id']).sum() # .unstack()['traf_halted_veh_num']
#time_stats_shot
dataset = pd.merge(edges, time_stats_shot, left_on='edge_id', right_on='edge_id')

In [48]:
stats['time']

0            0
1            0
2            0
3            0
4            0
5            0
6            0
7            0
8            0
9            0
10           0
11           0
12           0
13           0
14           0
15           0
16           0
17           0
18           0
19           0
20           0
21           0
22           0
23           0
24           0
25           0
26           0
27           0
28           0
29           0
          ... 
579316    5256
579317    5256
579318    5256
579319    5256
579320    5256
579321    5256
579322    5256
579323    5256
579324    5256
579325    5256
579326    5256
579327    5256
579328    5256
579329    5256
579330    5256
579331    5256
579332    5256
579333    5256
579334    5256
579335    5256
579336    5256
579337    5256
579338    5256
579339    5256
579340    5256
579341    5256
579342    5256
579343    5256
579344    5256
579345    5256
Name: time, Length: 579346, dtype: int64

## Part 3 : Draw the heatmap

In [38]:
# df=pd.read_csv('locations.csv', low_memory=False, index_col = 'key')  # read geocoded location from file
v_lat_long = dataset[['center_lat','center_long']]
v_weight = dataset['traf_halted_veh_num']

### Drawing the map
To call the function drawHeatMap, set up the parameters and call the function to draw the heatmap. You can experiment with different parameter values. The result map can also be saved as PNG file.

In [39]:
# set up parameters
zoom=13
max_intensity=float(np.max(v_weight))
radius=10

# call the function to draw the heatmap
drawHeatMap('Alcala de Henares', v_lat_long, v_weight, zoom, max_intensity, radius)

TraitError: The 'locations' trait of a WeightedHeatmap instance must be of length 1 <= L <= 9223372036854775807, but a value of [] was specified.